In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import gensim
import numpy as np
import pandas as pd

X_train=pd.read_csv('/kaggle/input/data-doc2vec/X_train.csv')
X_test=pd.read_csv('/kaggle/input/data-doc2vec/X_test.csv')
y_train=pd.read_csv('/kaggle/input/data-doc2vec/y_train.csv')
y_test=pd.read_csv('/kaggle/input/data-doc2vec/y_test.csv')

Create Doc2vec vectors

In [2]:
# Created TaggedDocument vectors for each text message in the training and test sets

tagged_docs_train = [gensim.models.doc2vec.TaggedDocument (v, [i]) for i,v in enumerate(X_train['clean_text'])]
tagged_docs_test = [gensim.models.doc2vec.TaggedDocument (v, [i]) for i,v in enumerate(X_test['clean_text'])]


In [3]:
tagged_docs_train[:10]

[TaggedDocument(words="['living', 'simple', 'loving', 'also', 'simple', 'laughing', 'simple', 'winning', 'tooo', 'simple', 'simple', 'difficult', '']", tags=[0]),
 TaggedDocument(words="['already', 'squatting', 'new', 'way', 'walking']", tags=[1]),
 TaggedDocument(words="['sister', 'got', 'placed', 'birla', 'soft', 'da']", tags=[2]),
 TaggedDocument(words="['lovely', 'smell', 'bus', 'aint', 'tobacco', '']", tags=[3]),
 TaggedDocument(words="['yes', 'nigh', 'cant', 'aha']", tags=[4]),
 TaggedDocument(words="['yup', 'remb', 'think', 'book', '']", tags=[5]),
 TaggedDocument(words="['sorry', 'roommates', 'took', 'forever', 'ok', 'come']", tags=[6]),
 TaggedDocument(words="['b4u', 'voucher', 'wc', '2703', 'marsms', 'log', 'onto', 'wwwb4utelecom', 'discount', 'credit', 'opt', 'reply', 'stop', 'customer', 'care', 'call', '08717168528']", tags=[7]),
 TaggedDocument(words="['kindly', 'send', 'one', 'flat', 'ltdecimalgt', 'today']", tags=[8]),
 TaggedDocument(words="['ceri', 'u', 'rebel', 'sweet

In [5]:
d2v_model = gensim.models.Doc2Vec(tagged_docs_train,
                                  vector_size=100,
                                  window=5,
                                  min_count=2)

In [6]:
# Infer the vectors to be used in training and testing

train_vectors= [d2v_model.infer_vector(eval(v.words)) for v in tagged_docs_train]

test_vectors = [d2v_model.infer_vector(eval(v.words)) for v in tagged_docs_test]   

Fit RandomForestClassifier On Top Of Document Vectors

In [7]:
# Fit a basic model, make predictions on the holdout test set, and the generate the evaluation metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score

rf = RandomForestClassifier()
rf_model = rf.fit(train_vectors, y_train.values.ravel())

y_pred = rf_model.predict(test_vectors)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test['label']).sum()/len(y_pred), 3)))

Precision: 0.937 / Recall: 0.401 / Accuracy: 0.917
